In [1]:
from pickle import load
import os
from random import choice
import time
# from agent_example import Agent
import vizdoom as vzd
from tqdm import tqdm

from DDQLAgent import DDQNAgent as Agent
from utils import StateWrapper
import pickle

import numpy as np

import torch as T
import torch.nn as nn
import torch.nn.functional as F

In [2]:
resolution = (1, 256, 192)

In [3]:
with open('new_framebuffer.pickle', 'rb') as f:
    memory = pickle.load(f)

In [5]:
states = memory.state_memory
variables = memory.vars_memory
actions = memory.action_memory[:, :7]

In [6]:
one_hot_actions = []
for a in actions:
    if a.sum() == 0:
        one_hot_actions.append(np.zeros(2**7))
        continue
    a = a.dot(1 << np.arange(a.size)[::-1])
    temp = np.zeros(2**7)
    temp[a] = 1
    break
    one_hot_actions.append(np.eye(2**7)[int(a)])

[0. 1. 0. 0. 0. 0. 0.]
32.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


In [7]:
print(np.eye(8))

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


In [23]:
d = np.array([9])
m = 7
test = np.flip((((d[:,None] & (1 << np.arange(m)))) > 0).astype(int))
print(test)

[[0 0 0 1 0 0 1]]


In [6]:
one_hot_actions = np.array(one_hot_actions)

In [ ]:
agent = Agent(input_dims=(1, resolution[1], resolution[2] * 3), n_actions=2**7, learning_rate=0.0001)
agent.q.train()

In [13]:
del memory

In [14]:
EPOCHS = 100
BATCH_SIZE = 128

In [15]:
for epoch in tqdm(range(EPOCHS)):
    indexes = np.arange(len(states))
    np.random.shuffle(indexes)
    for i in range(len(states)//BATCH_SIZE):
        batch_indexes = indexes[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
        batch_states = states[batch_indexes]
        batch_variables = variables[batch_indexes]
        batch_actions = one_hot_actions[batch_indexes]

        batch_states = T.tensor(batch_states, dtype=T.float).to(agent.q.device)
        batch_variables = T.tensor(batch_variables, dtype=T.float).to(agent.q.device)
        batch_actions = T.tensor(batch_actions, dtype=T.float).to(agent.q.device)

        agent.q.optimizer.zero_grad()
        preds = agent.q(batch_states, batch_variables)
        loss = agent.q.loss(preds, batch_actions).to(agent.q.device)
        loss.backward()
        agent.q.optimizer.step()

  0%|          | 0/100 [00:28<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# agent.update_weights(tau=1)

In [ ]:
os.makedirs(f"tmp/ddql/fitted", exist_ok=True)
agent.save_model(path='./tmp/ddql/fitted')